In [1]:
!pip install konlpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 69.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.8/493.8 kB 30.7 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
from konlpy.tag import Okt
import pandas as pd

# 형태소 분석기 객체 생성
okt = Okt()

# CSV 파일 읽기 (파일 경로에 맞게 수정)
file_path = '/content/drive/MyDrive/신흥반점_전처리(P2C_메뉴버전).csv'
df = pd.read_csv(file_path)

# 첫 5개 행 출력
df.head()


,매장명,닉네임,review_spell_check,label,메뉴
0,24시 신흥반점,cg**,너무 맛나게 먹었습니다,1.0,누룽지탕/1
1,24시 신흥반점,de**,양도 많고 맛있습니다 자주 주문할 것 같아요,1.0,"등심탕수육/1(사이즈 선택(小),소스 선택(간장소스)),짬짜면/1(곱빼기 선택(기본))"
2,24시 신흥반점,to**,오랜만에 보는 옛날 탕수육이라 좋았어요,1.0,등심탕수육＋짜장2/1(소스 선택(간장소스))
3,24시 신흥반점,le**,탕볶밥에 군만두 너무너무 맛있었어요 탕수육 볶음밥 군만두까지 다 맛있었던,1.0,"탕볶밥/1(곱빼기 선택(기본)),수제 군만두/1"
4,24시 신흥반점,go**,전날 술 먹고 어질한데 먹고 살아났습니다 얼큰하고 시원해서 좋네요,1.0,"속풀이짬뽕/1(곱빼기 선택(기본)),물만두/1"


In [5]:
# 'label' 열과 '메뉴' 열만 선택
df_selected = df[['label', 'review_spell_check','메뉴']]

# 결과 확인
df_selected.head()


,label,review_spell_check,메뉴
0,1.0,너무 맛나게 먹었습니다,누룽지탕/1
1,1.0,양도 많고 맛있습니다 자주 주문할 것 같아요,"등심탕수육/1(사이즈 선택(小),소스 선택(간장소스)),짬짜면/1(곱빼기 선택(기본))"
2,1.0,오랜만에 보는 옛날 탕수육이라 좋았어요,등심탕수육＋짜장2/1(소스 선택(간장소스))
3,1.0,탕볶밥에 군만두 너무너무 맛있었어요 탕수육 볶음밥 군만두까지 다 맛있었던,"탕볶밥/1(곱빼기 선택(기본)),수제 군만두/1"
4,1.0,전날 술 먹고 어질한데 먹고 살아났습니다 얼큰하고 시원해서 좋네요,"속풀이짬뽕/1(곱빼기 선택(기본)),물만두/1"


In [6]:
import pandas as pd
import re  # 정규 표현식을 사용하여 괄호 안의 내용 제거

# 사용자 지정 메뉴 리스트 (예시)
menu_keywords = ['짜장', '삼선간짜장', '속풀이짬뽕', '차돌짬뽕', '중화비빔밥', '유린기', '알짬뽕', '낙지짬뽕', '고기짜장', '후라이짜장', '얼큰짜장', '짬뽕', '굴짬뽕',
                 '삼선짬뽕', '육해공짬뽕', '우동', '삼선우동', '울면', '삼선울면', '수제비짬뽕', '사천탕면', '간짜장', '불고기덮밥', '볶음밥', '김치볶음밥', '삼선볶음밥', '새우볶음밥', '고기볶음밥',
                 '제육덮밥', '오징어덮밥', '송이덯밥', '잡채덮밥', '고추잡채밥', '유산슬밥', '잡탕밥', '공기밥','육개장','떡만두국','만두국',
                 '갈비탕', '수제비짬뽕밥','내장탕', '마파두부밥', '낙지덮밥', '게살볶음밥', '미역국', '등심탕수육+짜장1','등심탕수육+짬뽕1','등심탕수육+짜장2'
                 ,'등심탕수육+짜장1+짬뽕1','등심탕수육+짬뽕2','등심탕수육+쟁반짜장2人', '등심탕수육+삼선짬뽕2','등심탕수육+속풀이짬뽕2',
                 '물냉면', '비빔냉면','열무냉면', '콩국수', '나혼냉세트A','나혼냉세트B','나혼냉세트C','커플세트1','커플세트2', '커플세트3','해물떡볶이',
                 '등심탕수육', '고추잡채','깐풍육','깐풍기','깐쇼새우','깐풍새우','크림새우','양장피','팔보채','해물쟁반짜장','해물볶음짬뽕','유산슬','마파두부요리','술국',
                 '누룽지탕','잡채','두가지 요리안주','짬짜면','볶짜면','볶짬면','탕짜면','탕짬면','탕볶밥','탕정식','똔까스+밥+짜장','수제 군만두','물만두','왕만두','튀김꽃빵']  # 사용자가 지정한 메뉴

# 제외할 단어 리스트 (사전에 정의된 단어)
exclude_keywords = ['소스 선택)']

# 메뉴 열에서 세부사항을 제거하고, 키워드만 추출하는 함수 정의
def preprocess_menu(menu):
    # 쉼표로 나누어 여러 메뉴가 있을 경우 처리
    menus = menu.split(',')

    cleaned_menus = []
    for item in menus:
        # '/'로 나누어 첫 번째 부분만 사용
        item = item.split('/')[0]

        # 괄호 안의 내용 제거 (정규표현식 사용)
        item = re.sub(r'\(.*?\)', '', item)

        # 불필요한 공백 제거
        item = item.strip()

        # 제외할 키워드가 포함된 메뉴를 제외
        if any(exclude in item for exclude in exclude_keywords):
            continue

        # 전처리된 메뉴 추가
        cleaned_menus.append(item)

    return cleaned_menus

# 'label' 열과 '메뉴' 열만 선택
df_selected = df[['label', 'review_spell_check','메뉴']]

# 메뉴 열에서 전처리 진행
df_selected['메뉴_전처리'] = df_selected['메뉴'].apply(preprocess_menu)

# 결과 확인
df_selected[['label', 'review_spell_check','메뉴', '메뉴_전처리']]


,label,review_spell_check,메뉴,메뉴_전처리
0,1.0,너무 맛나게 먹었습니다,누룽지탕/1,[누룽지탕]
1,1.0,양도 많고 맛있습니다 자주 주문할 것 같아요,"등심탕수육/1(사이즈 선택(小),소스 선택(간장소스)),짬짜면/1(곱빼기 선택(기본))","[등심탕수육, 짬짜면]"
2,1.0,오랜만에 보는 옛날 탕수육이라 좋았어요,등심탕수육＋짜장2/1(소스 선택(간장소스)),[등심탕수육＋짜장2]
3,1.0,탕볶밥에 군만두 너무너무 맛있었어요 탕수육 볶음밥 군만두까지 다 맛있었던,"탕볶밥/1(곱빼기 선택(기본)),수제 군만두/1","[탕볶밥, 수제 군만두]"
4,1.0,전날 술 먹고 어질한데 먹고 살아났습니다 얼큰하고 시원해서 좋네요,"속풀이짬뽕/1(곱빼기 선택(기본)),물만두/1","[속풀이짬뽕, 물만두]"
...,...,...,...,...
849,0.0,중국집에서 양파춘 장 없이 딸랑 단무지만 비닐봉지에 받아보기는 첨입니다,"후라이짜장/1(곱빼기 선택(곱빼기)),짬뽕/1(곱빼기 선택(기본))","[후라이짜장, 짬뽕]"
850,0.0,배달 진심 너무 느렸어요 배달은 별 0개 드리고 싶어요,"비빔냉면/1(곱빼기 선택(기본)),해물볶음짬뽕/1(사이즈 선택(1인))","[비빔냉면, 해물볶음짬뽕]"
851,0.0,바쁘셔서 예상 배달 시간보다 10분 정도 더 기다렸는데전체적으로 맛있어요하지만 공깃...,"후라이짜장/1(곱빼기 선택(기본)),짬뽕/1(곱빼기 선택(기본)),공기밥/1","[후라이짜장, 짬뽕, 공기밥]"
852,0.0,제육에서 불향이 아니라 그을린 냄새가 너무 나서전화하려다 말았습니다,"제육덮밥/1(곱빼기 선택(기본)),탕볶밥/1(곱빼기 선택(기본))","[제육덮밥, 탕볶밥]"


In [ ]:
import pandas as pd
from collections import Counter

# 메뉴 전처리 함수가 이미 적용된 df_selected
# 각 메뉴별 긍정과 부정 비율을 계산

# 각 메뉴별로 긍정(1)과 부정(0)의 비율 계산
def calculate_positive_probabilities(df):
    menu_probabilities = {}

    # 각 메뉴에 대해 긍정/부정 비율 계산
    for _, row in df.iterrows():
        # 메뉴에 대한 리스트
        menus = row['메뉴_전처리']
        label = row['label']

        for menu in menus:
            # 이미 있는 메뉴는 그 값에 +1 추가
            if menu in menu_probabilities:
                menu_probabilities[menu]['positive'] += label
                menu_probabilities[menu]['total'] += 1
            else:
                menu_probabilities[menu] = {'positive': label, 'total': 1}

    # 각 메뉴의 긍정 확률 계산
    for menu, values in menu_probabilities.items():
        positive_prob = values['positive'] / values['total']
        menu_probabilities[menu]['positive_probability'] = positive_prob

    return menu_probabilities

# 메뉴별 긍정 확률 계산
menu_probabilities = calculate_positive_probabilities(df_selected)

# 메뉴별 출현 빈도 계산
menu_frequency = Counter([menu for sublist in df_selected['메뉴_전처리'] for menu in sublist])

# 출현 빈도가 높은 상위 10개 메뉴 선택
top_10_menus = [menu for menu, _ in menu_frequency.most_common(20)]

# 상위 10개 메뉴에 대한 긍정 확률 출력
print("상위 10개 메뉴의 긍정 확률:")
for menu in top_10_menus:
    if menu in menu_probabilities:
        positive_prob = menu_probabilities[menu]['positive_probability']
        print(f"{menu}: 긍정 확률 = {positive_prob:.2f}")
    else:
        print(f"{menu}: 긍정 확률 정보 없음")

# 사용자 입력에 대한 긍정 확률 계산
def get_positive_probability_for_input(menu_input, menu_probabilities):
    menu_input = menu_input.strip()
    if menu_input in menu_probabilities:
        return menu_probabilities[menu_input]['positive_probability']
    else:
        return None  # 메뉴가 데이터셋에 없다면


In [ ]:
user_input = input("메뉴를 입력하세요: ")
positive_probability = get_positive_probability_for_input(user_input, menu_probabilities)

if positive_probability is not None:
    print(f"{user_input}에 대한 긍정 확률은 {positive_probability:.2f}입니다.")
else:
    print(f"입력하신 메뉴 '{user_input}'는 데이터셋에 존재하지 않습니다.")


In [9]:
# 사용자 입력에 대한 리뷰 필터링 함수
def get_reviews_for_menu(menu_input, df):
    # 메뉴 입력값을 처리하여 공백 제거
    menu_input = menu_input.strip()

    # 해당 메뉴가 포함된 리뷰들 필터링 (메뉴_전처리 열에 해당 메뉴가 포함된 행들 선택)
    filtered_reviews = df[df['메뉴_전처리'].apply(lambda x: menu_input in x)]

    # 결과로 'review_spell_check' 열만 출력
    return filtered_reviews['review_spell_check']

# 사용자 입력받기
user_input = input("메뉴를 입력하세요: ")

# 해당 메뉴에 대한 리뷰 출력
reviews = get_reviews_for_menu(user_input, df_selected)

# 리뷰 출력 (해당 메뉴가 포함된 리뷰들)
if not reviews.empty:
    print(f"{user_input}에 해당하는 리뷰들:")
    print(reviews)
else:
    print(f"입력하신 메뉴 '{user_input}'에 해당하는 리뷰가 없습니다.")


메뉴를 입력하세요: 짬뽕
짬뽕에 해당하는 리뷰들:
6                              사장님 친절하고 맛도 좋아요 서비스 감사합니다
12                                           맛있게 잘 먹었습니다
27                                              잘 먹었습니다 
62                                      맛있게 잘 먹었습니다 다음에도
158                                         탕수육 많고 맛있었어요
169    군만두 서비스 감사합니다♡ 단무지 요청 들어주시고 감사합니다_x000D_언제나 짬뽕...
174    맛나게 먹었어요_x000D_날씨가 추워서 식은 게 좀 아쉽지만 _x000D_날씨 탓...
178                                       맛있어요 사장님 번창하세요
185               요청 사항 항상 다 들어주셔서 감사합니다_x000D_잘 먹고 있습니다
196                           짬뽕밥 먹고 싶어서 시켰는데 먹을 만하네요 굿굿
213                                      항상 맛있게 잘 먹고 있어요
220                                         맛있게 잘 먹었습니다_
221                    빠르게 와서 잘 먹었습니다_x000D__x000D_감사합니다
230                                            맛있게잘먹었슺니다
251                                      배달도 빠르고 잘 먹었습니다
252                                   배달도 빠르고 양도 많고 맛있네요
260                          맛있습니다 다른 중국집 가지 마시고 여기로 오세요
279